In [1]:
import numpy as np
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import re
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df1 = pd.read_csv('../input/questionanswer-dataset/S08_question_answer_pairs.txt', sep='\t')
df2 = pd.read_csv('../input/questionanswer-dataset/S09_question_answer_pairs.txt', sep='\t')
df3 = pd.read_csv('../input/questionanswer-dataset/S10_question_answer_pairs.txt', sep='\t', encoding = 'ISO-8859-1')

In [3]:
df1.head(4)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4


In [4]:
df2.head(3)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,S09_set4_a10
1,Alessandro_Volta,Was Volta an Italian physicist?,yes,easy,easy,S09_set4_a10
2,Alessandro_Volta,Is Volta buried in the city of Pittsburgh?,no,easy,easy,S09_set4_a10


In [5]:
data=df1.append([df2,df3])

In [6]:
data.head(4)

,ArticleTitle,Question,Answer,DifficultyFromQuestioner,DifficultyFromAnswerer,ArticleFile
0,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,yes,easy,easy,S08_set3_a4
1,Abraham_Lincoln,Was Abraham Lincoln the sixteenth President of...,Yes.,easy,easy,S08_set3_a4
2,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,yes,easy,medium,S08_set3_a4
3,Abraham_Lincoln,Did Lincoln sign the National Banking Act of 1...,Yes.,easy,easy,S08_set3_a4


In [7]:
#here we used only improtent column for the automatic question answering inly we need question and answer column 
data=data[['Question','Answer']]

In [8]:
data.head(15)

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
1,Was Abraham Lincoln the sixteenth President of...,Yes.
2,Did Lincoln sign the National Banking Act of 1...,yes
3,Did Lincoln sign the National Banking Act of 1...,Yes.
4,Did his mother die of pneumonia?,no
5,Did his mother die of pneumonia?,No.
6,How many long was Lincoln's formal education?,18 months
7,How many long was Lincoln's formal education?,18 months.
8,When did Lincoln begin his political career?,1832
9,When did Lincoln begin his political career?,1832.


In [9]:
data.shape

(3998, 2)

In [10]:
#remove duplicate column 
data =data.drop_duplicates(subset='Question')
data.head(10)

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832
10,What did The Legal Tender Act of 1862 establish?,"the United States Note, the first paper curren..."
12,Who suggested Lincoln grow a beard?,11-year-old Grace Bedell
14,When did the Gettysburg address argue that Ame...,1776
16,Did Lincoln beat John C. Breckinridge in the 1...,yes
18,Was Abraham Lincoln the first President of the...,No


In [11]:
data.shape

(2457, 2)

In [12]:
data=data.dropna()

In [13]:
data.head(10)

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832
10,What did The Legal Tender Act of 1862 establish?,"the United States Note, the first paper curren..."
12,Who suggested Lincoln grow a beard?,11-year-old Grace Bedell
14,When did the Gettysburg address argue that Ame...,1776
16,Did Lincoln beat John C. Breckinridge in the 1...,yes
18,Was Abraham Lincoln the first President of the...,No


In [14]:
data.shape

(2185, 2)

# Automatic Question Answering 
# Convert each query question to vector
# Find the closet question Vector to uses question vector
# *used cosine simlarity*
# ***Convert Word to Vector Following Method***
* Bag of Word
* word to vec
* Glove
* Bert
# Data Preprocessing 
# here we are using only bag of word method to 

In [15]:
data.head(5)

,Question,Answer
0,Was Abraham Lincoln the sixteenth President of...,yes
2,Did Lincoln sign the National Banking Act of 1...,yes
4,Did his mother die of pneumonia?,no
6,How many long was Lincoln's formal education?,18 months
8,When did Lincoln begin his political career?,1832


In [16]:
#using stop word to clean the data
def clean_sentence(sentence,stopwords=False):
    sentence=sentence.lower().strip()
    sentence=re.sub(r'[^a-z0-9\s]',' ',sentence)
    if stopwords:
        sentence=remove_stopwords(sentence)
    return sentence

In [17]:
def get_cleaned_sentence(data,stopwords=False):
    sents=data[['Question']];
    cleaned_sentence=[]
    for index,row in data.iterrows():
        cleaned=clean_sentence(row['Question'],stopwords)
        cleaned_sentence.append(cleaned);
    return cleaned_sentence
cleaned_sentence=get_cleaned_sentence(data,stopwords=True)
print(cleaned_sentence);
print('\n')

['abraham lincoln sixteenth president united states', 'lincoln sign national banking act 1863', 'mother die pneumonia', 'long lincoln s formal education', 'lincoln begin political career', 'legal tender act 1862 establish', 'suggested lincoln grow beard', 'gettysburg address argue america born', 'lincoln beat john c breckinridge 1860 election', 'abraham lincoln president united states', 'lincoln start political career 1832', 'lincoln represent alton sangamon railroad', 'county lincoln born', 'lincoln serve president', 'assassinated lincoln', 'lincoln win election 1860', 'general charge battle antietam', 'lincoln issue emancipation proclamation', 'noted contributions theory molarity molecular weight', 'graduated ecclesiastical law early age 20 began practice', 'publish memoria', 'professor', 'true professor 1820', 'lorenzo romano amedeo carlo avogadro italian savant', 'amedeo avogadro born turin august 9th 1776 noble ancient family piedmont italy', 'noted contributions theory molarity m

In [18]:
#withut using the stop words
clean_sentence_with_stopwords=get_cleaned_sentence(data,stopwords=False)
print(clean_sentence_with_stopwords)

['was abraham lincoln the sixteenth president of the united states ', 'did lincoln sign the national banking act of 1863 ', 'did his mother die of pneumonia ', 'how many long was lincoln s formal education ', 'when did lincoln begin his political career ', 'what did the legal tender act of 1862 establish ', 'who suggested lincoln grow a beard ', 'when did the gettysburg address argue that america was born ', 'did lincoln beat john c  breckinridge in the 1860 election ', 'was abraham lincoln the first president of the united states ', 'did lincoln start his political career in 1832 ', 'did lincoln ever represent alton   sangamon railroad ', 'which county was lincoln born in ', 'when did lincoln first serve as president ', 'who assassinated lincoln ', 'did lincoln win the election of 1860 ', 'who was the general in charge at the battle of antietam ', 'why did lincoln issue the emancipation proclamation ', 'who is most noted for his contributions to the theory of molarity and molecular we

In [19]:
#convert sentences to word
#here we usiing stop bag of word method to predict the sentences
sentences=clean_sentence_with_stopwords
#split the word with white spaces
sentence_words=[[word for word in  document.split()]for document in sentences]
#convert the word to the dictonery
dictionary=corpora.Dictionary(sentence_words)
for key,value in dictionary.items():
    print(key,':',value)

0 : abraham
1 : lincoln
2 : of
3 : president
4 : sixteenth
5 : states
6 : the
7 : united
8 : was
9 : 1863
10 : act
11 : banking
12 : did
13 : national
14 : sign
15 : die
16 : his
17 : mother
18 : pneumonia
19 : education
20 : formal
21 : how
22 : long
23 : many
24 : s
25 : begin
26 : career
27 : political
28 : when
29 : 1862
30 : establish
31 : legal
32 : tender
33 : what
34 : a
35 : beard
36 : grow
37 : suggested
38 : who
39 : address
40 : america
41 : argue
42 : born
43 : gettysburg
44 : that
45 : 1860
46 : beat
47 : breckinridge
48 : c
49 : election
50 : in
51 : john
52 : first
53 : 1832
54 : start
55 : alton
56 : ever
57 : railroad
58 : represent
59 : sangamon
60 : county
61 : which
62 : as
63 : serve
64 : assassinated
65 : win
66 : antietam
67 : at
68 : battle
69 : charge
70 : general
71 : emancipation
72 : issue
73 : proclamation
74 : why
75 : and
76 : contributions
77 : for
78 : is
79 : molarity
80 : molecular
81 : most
82 : noted
83 : theory
84 : to
85 : weight
86 : 20
87 : age

786 : related
787 : 38th
788 : leader
789 : minority
790 : eagle
791 : scout
792 : district
793 : elected
794 : funeral
795 : procession
796 : scouts
797 : future
798 : heisman
799 : tackle
800 : trophy
801 : winner
802 : extra
803 : need
804 : 90
805 : five
806 : lived
807 : past
808 : eniwetok
809 : kwajalein
810 : 13
811 : 1942
812 : hospital
813 : released
814 : grand
815 : interred
816 : museum
817 : presidential
818 : rapids
819 : former
820 : four
821 : commission
822 : last
823 : surviving
824 : warren
825 : bartel
826 : incumbent
827 : j
828 : jonkman
829 : supporters
830 : take
831 : urge
832 : claim
833 : exchange
834 : nixon
835 : pardon
836 : pro
837 : quid
838 : quo
839 : resignation
840 : capita
841 : ghana
842 : output
843 : per
844 : poorer
845 : twice
846 : build
847 : castle
848 : dutch
849 : elmina
850 : achieve
851 : independence
852 : kingdom
853 : divided
854 : head
855 : dominant
856 : religion
857 : ghanaian
858 : native
859 : subfamilies
860 : lake
861 : volta

1452 : rules
1453 : plays
1454 : principality
1455 : available
1456 : balzers
1457 : charter
1458 : flights
1459 : helicopter
1460 : heliport
1461 : liechtensteiners
1462 : nationals
1463 : plural
1464 : although
1465 : haven
1466 : perhaps
1467 : resort
1468 : tax
1469 : austria
1470 : cases
1471 : feudal
1472 : fief
1473 : lords
1474 : lower
1475 : moravia
1476 : senior
1477 : silesia
1478 : styria
1479 : these
1480 : though
1481 : i
1482 : nights
1483 : sleep
1484 : webster
1485 : write
1486 : arrival
1487 : exiled
1488 : fillmore
1489 : hungarian
1490 : itself
1491 : kossuth
1492 : louis
1493 : presented
1494 : 10
1495 : 11
1496 : 1874
1497 : 8
1498 : effects
1499 : m
1500 : p
1501 : stroke
1502 : got
1503 : senate
1504 : seward
1505 : commodore
1506 : japan
1507 : matthew
1508 : perry
1509 : western
1510 : millard
1511 : remarkable
1512 : buffalo
1513 : partnership
1514 : private
1515 : caroline
1516 : mcintosh
1517 : 7
1518 : january
1519 : 1974
1520 : love
1521 : disagree
1522 :

2119 : brand
2120 : crow
2121 : whiskey
2122 : smoked
2123 : sporadically
2124 : farmers
2125 : galena
2126 : harnesses
2127 : hides
2128 : leather
2129 : perkins
2130 : prosperous
2131 : purchase
2132 : saddles
2133 : bush
2134 : corner
2135 : dragon
2136 : entrance
2137 : gate
2138 : 1865
2139 : northwesten
2140 : uruguay
2141 : montevideo
2142 : summers
2143 : oldest
2144 : architecture
2145 : influenced
2146 : informally
2147 : neighborhoods
2148 : poor
2149 : landscape
2150 : mountainous
2151 : uruguayans
2152 : citizens
2153 : propose
2154 : cup
2155 : host
2156 : kilometres
2157 : descent
2158 : times
2159 : compared
2160 : income
2161 : man
2162 : woman
2163 : recognize
2164 : sex
2165 : unions
2166 : guiana
2167 : june
2168 : month
2169 : warmest
2170 : citizen
2171 : competitive
2172 : done
2173 : names
2174 : parties
2175 : soverign
2176 : stronghold
2177 : meanings
2178 : india
2179 : 1828
2180 : temperate
2181 : sport
2182 : carlos
2183 : maldonado
2184 : san
2185 : agricu

2785 : someone
2786 : you
2787 : carbon
2788 : chlorine
2789 : compound
2790 : synthesis
2791 : diamagnetism
2792 : phenomenon
2793 : cartoon
2794 : foul
2795 : oft
2796 : punch
2797 : reprinted
2798 : resulted
2799 : thames
2800 : wrote
2801 : bulk
2802 : colloids
2803 : corresponding
2804 : differed
2805 : optical
2806 : properties
2807 : those
2808 : assistant
2809 : davy
2810 : hampton
2811 : butts
2812 : newington
2813 : foundation
2814 : motor
2815 : technology
2816 : denomination
2817 : maine
2818 : fulton
2819 : operation
2820 : placed
2821 : robert
2822 : coliseum
2823 : arena
2824 : dead
2825 : fighting
2826 : modeled
2827 : town
2828 : base
2829 : edison
2830 : hired
2831 : communication
2832 : demonstrate
2833 : radio
2834 : wireless
2835 : density
2836 : flux
2837 : induction
2838 : measuring
2839 : mad
2840 : regarded
2841 : scientist
2842 : currents
2843 : ottawa
2844 : ottowa
2845 : sit
2846 : tremors
2847 : industries
2848 : junior
2849 : athletic
2850 : carleton
2851 

3451 : commanded
3452 : sailed
3453 : voyage
3454 : migration
3455 : overcrowding
3456 : poverty
3457 : stem
3458 : fortress
3459 : jayawikarta
3460 : suharto
3461 : indianized
3462 : tarumanagara
3463 : istiqlal
3464 : landmarks
3465 : mosque
3466 : monuments
3467 : sukarno
3468 : cultural
3469 : job
3470 : memorial
3471 : 20th
3472 : koalas
3473 : metabolism
3474 : slow
3475 : eucalypt
3476 : forests
3477 : rainforests
3478 : ecological
3479 : fill
3480 : koala
3481 : role
3482 : sloth
3483 : therefore
3484 : viability
3485 : weakness
3486 : adelaide
3487 : australia
3488 : cape
3489 : coastal
3490 : enough
3491 : inland
3492 : moisture
3493 : suitable
3494 : woodlands
3495 : exterminated
3496 : repopulated
3497 : victorian
3498 : korea
3499 : goryeo
3500 : koreans
3501 : adjectives
3502 : misleadingly
3503 : translation
3504 : verbs
3505 : loanwords
3506 : sino
3507 : speech
3508 : paradigms
3509 : verb
3510 : numbers
3511 : columns
3512 : rows
3513 : estimate
3514 : jae
3515 : jeon

In [20]:
bow=[dictionary.doc2bow(text) for text in sentence_words]
for sent,embedding in zip(sentences,bow):
    print(sent)
    print(embedding)

was abraham lincoln the sixteenth president of the united states 
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 2), (7, 1), (8, 1)]
did lincoln sign the national banking act of 1863 
[(1, 1), (2, 1), (6, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]
did his mother die of pneumonia 
[(2, 1), (12, 1), (15, 1), (16, 1), (17, 1), (18, 1)]
how many long was lincoln s formal education 
[(1, 1), (8, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1)]
when did lincoln begin his political career 
[(1, 1), (12, 1), (16, 1), (25, 1), (26, 1), (27, 1), (28, 1)]
what did the legal tender act of 1862 establish 
[(2, 1), (6, 1), (10, 1), (12, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)]
who suggested lincoln grow a beard 
[(1, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)]
when did the gettysburg address argue that america was born 
[(6, 1), (8, 1), (12, 1), (28, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1)]
did lincoln beat john c  breckinridge in the 1860 e

loschmidt calculated first the value of what 
[(2, 1), (6, 1), (33, 1), (52, 1), (2293, 1), (2300, 1), (2301, 1)]
hence   can relative molecular masses be calculated from the masses of gas samples  
[(2, 1), (6, 1), (80, 1), (142, 1), (143, 1), (205, 1), (987, 1), (2293, 1), (2294, 1), (2295, 2), (2296, 1), (2297, 1)]
can the title of this famous 1811 paper be roughly translated into english as  essay on determining the relative masses of the elementary molecules of bodies   
[(2, 3), (6, 3), (62, 1), (141, 1), (142, 1), (143, 1), (144, 1), (145, 1), (146, 1), (147, 1), (148, 1), (149, 1), (150, 1), (151, 1), (186, 1), (2295, 1), (2296, 1), (2302, 1), (2303, 1), (2304, 1), (2305, 1), (2306, 1)]
what is amedeo avogadro 
[(33, 1), (78, 1), (104, 1), (106, 1)]
where is amedeo avogadro from 
[(78, 1), (104, 1), (106, 1), (205, 1), (359, 1)]
when did anders celcius publish his observations on the aurora borealis 
[(6, 1), (12, 1), (16, 1), (28, 1), (97, 1), (179, 1), (186, 1), (2307, 1), (2

is the violin cello a bowed string instrument 
[(6, 1), (34, 1), (78, 1), (2401, 1), (2402, 1), (2403, 1), (2404, 1), (2405, 1)]
is the purfling just for decoration 
[(6, 1), (77, 1), (78, 1), (2406, 1), (2407, 1), (2408, 1)]
are the z holes used as access points to the interior of the cello 
[(2, 1), (6, 3), (62, 1), (84, 1), (134, 1), (207, 1), (981, 1), (2052, 1), (2402, 1), (2409, 1), (2410, 1), (2411, 1)]
what does violincello mean 
[(33, 1), (438, 1), (555, 1), (2412, 1)]
what are the two materials that bows are made of traditionally 
[(2, 1), (6, 1), (33, 1), (44, 1), (207, 2), (390, 1), (547, 1), (2413, 1), (2414, 1), (2415, 1)]
what is used to tune a cello 
[(33, 1), (34, 1), (78, 1), (84, 1), (134, 1), (2402, 1), (2416, 1)]
which harmonics can produce any notes above middle c 
[(48, 1), (61, 1), (143, 1), (600, 1), (1291, 1), (2417, 1), (2418, 1), (2419, 1), (2420, 1)]
who referred to the cello as  basso de viola da braccio  
[(6, 1), (38, 1), (62, 1), (84, 1), (408, 1), (128

does the koala fill the same ecological role as the sloth of south america 
[(2, 1), (6, 3), (40, 1), (62, 1), (438, 1), (713, 1), (1853, 1), (3478, 1), (3479, 1), (3480, 1), (3481, 1), (3482, 1)]
is the long term viability of the koala therefore threatened by genetic weakness 
[(2, 1), (6, 2), (22, 1), (78, 1), (274, 1), (1831, 1), (2060, 1), (3320, 1), (3480, 1), (3483, 1), (3484, 1), (3485, 1)]
is the koala found in coastal regions of eastern and southern australia  from near adelaide to the southern part of cape york peninsula 
[(2, 2), (6, 2), (50, 1), (75, 1), (78, 1), (84, 1), (205, 1), (210, 1), (212, 1), (444, 1), (608, 1), (922, 1), (1642, 2), (1978, 1), (2540, 1), (3480, 1), (3486, 1), (3487, 1), (3488, 1), (3489, 1)]
do populations also extend for considerable distances inland in regions with enough moisture to support suitable woodlands 
[(50, 1), (77, 1), (84, 1), (160, 1), (212, 1), (214, 1), (660, 1), (666, 1), (925, 1), (979, 1), (1294, 1), (2709, 1), (3490, 1), (3491,

[(2, 1), (6, 2), (8, 1), (62, 1), (77, 1), (102, 1), (134, 1), (186, 1), (257, 1), (314, 1), (391, 1), (1943, 1), (3287, 1), (3288, 1), (3620, 1), (3621, 1), (3870, 1)]
is the city named after the apostle saint peter 
[(6, 2), (78, 1), (161, 1), (163, 1), (727, 1), (2888, 1), (3871, 1), (3872, 1)]
has the terrain in the city been artificially raised 
[(6, 2), (50, 1), (384, 1), (420, 1), (727, 1), (1197, 1), (3873, 1), (3874, 1)]
what were some other names of the city 
[(2, 1), (6, 1), (33, 1), (238, 1), (360, 1), (558, 1), (727, 1), (2173, 1)]
what festival features fireworks  celebrating the end of the school year 
[(2, 1), (6, 2), (33, 1), (375, 1), (383, 1), (1057, 1), (2932, 1), (3875, 1), (3876, 1), (3877, 1)]
where do most people in urban saint petersburg live 
[(50, 1), (81, 1), (214, 1), (359, 1), (607, 1), (719, 1), (2888, 1), (3878, 1), (3879, 1)]
what was made in russia s largest foundry 
[(8, 1), (24, 1), (33, 1), (50, 1), (447, 1), (547, 1), (3880, 1), (3881, 1)]
when did

In [21]:
#Bag of word represent for Questions
Question_orig='When Abraham Lincoln started his political career'
question=clean_sentence(Question_orig,stopwords=False)
question_embedding=dictionary.doc2bow(question.split())
print('\n\n',question,'\n',question_embedding)



 when abraham lincoln started his political career 
 [(0, 1), (1, 1), (16, 1), (26, 1), (27, 1), (28, 1)]


Convert vector represnt to the cell using bag of words after here find distance between to vector using cosine simlarity closet matching answer can be find the cosine simlarity  we saw the bow are not gave the quality of questions

In [22]:
def nearest_question(question_embedding,sentence_embedding,ques,sentences):
    max_simalirty=-1;
    index_simalrit=-1;
    for index,answer_embedding in enumerate(sentence_embedding):
        simlarity=cosine_similarity(answer_embedding,question_embedding)[0][0];
        print(index,simlarity,sentences[index])
        #which question max simalryt and sort them
        if simlarity>max_simalirty:
            max_simalirty=simlarity;
            index_simalrity=index;
    print('\n')
    print('question',question)
    print('\n')
    print('retrived',ques.iloc[index_simalrity,0])
    print(ques.iloc[index_simalrity,1])
nearest_question(question_embedding,bow,data,sentences)

0 1.0 was abraham lincoln the sixteenth president of the united states 
1 0.7071067811865475 did lincoln sign the national banking act of 1863 
2 0.4472135954999579 did his mother die of pneumonia 
3 0.7071067811865475 how many long was lincoln s formal education 
4 0.7071067811865475 when did lincoln begin his political career 
5 0.4472135954999579 what did the legal tender act of 1862 establish 
6 0.7071067811865475 who suggested lincoln grow a beard 
7 0.1643989873053573 when did the gettysburg address argue that america was born 
8 0.7071067811865475 did lincoln beat john c  breckinridge in the 1860 election 
9 1.0 was abraham lincoln the first president of the united states 
10 0.7071067811865475 did lincoln start his political career in 1832 
11 0.7071067811865475 did lincoln ever represent alton   sangamon railroad 
12 0.7071067811865475 which county was lincoln born in 
13 0.7071067811865475 when did lincoln first serve as president 
14 0.7071067811865475 who assassinated linco

140 0.004672846178479892 do all ducks  quack  
141 0.7071067811865475 what is the name of a specialized species of duck adapted to catch large fish 
142 0.1643989873053573 what animal attracts the most humor and silliness 
143 0.4472135954999579 what is an economic use of a duck 
144 0.02630668208823282 who is daffy duck 
145 0.030289126640769135 what makes it more difficult for a diving duck to fly 
146 0.1643989873053573 what are the reasons for hunting wild ducks 
147 0.004672846178479892 do ducks forage underwater 
148 0.01281945932506583 is foie gras often made from ducks 
149 0.030289126640769135 what economic uses to ducks have 
150 0.4472135954999579 what kind of ducks feed on land 
151 0.030289126640769135 what unrelated water birds are ducks sometimes confused with 
152 0.1643989873053573 what does the word duck mean 
153 0.4472135954999579 what are some common predators of ducks 
154 0.047565149415449405 how does a mallard s tongue work 
155 0.004830861503963699 are diving d

332 0.1643989873053573 who won the 1903 nobel prize in physics for discovering radioactivity  
333 0.4472135954999579 did he not die at the age of 55 in le croisic  
334 0.4472135954999579 did one places between the phosphorescent substance and the paper a piece of money or a metal screen not pierce with a cut out design  
335 0.030289126640769135 what happened in 1896 
336 0.4472135954999579 what happened with a bromide emulsion in two sheets of very thick black paper 
337 0.1643989873053573 is there a becquerel crater on the moon and a becquerel crater on mars 
338 0.31622776601683794 are the javanese the largest and politically dominant ethnic group in indonesia 
339 0.058722021951470346 is indonesia a monarchy with a presidential system 
340 0.1643989873053573 did the japanese invasion and subsequent occupation during wwii end dutch rule 
341 0.08304547985373997 in what year did east timor secede from indonesia 
342 0.1643989873053573 when did islam become the dominant religion in 

447 0.4472135954999579 what is the average life expectancy of a kangaroo 
448 0.4472135954999579 when did the first official report of kangaroo blindness take place 
449 0.030289126640769135 what are vehicles that frequent isolated roads often fitted with 
450 0.004672846178479892 do kangaroos eat plants 
451 0.4472135954999579 what are some extinct predators of the kangaroo 
452 0.1643989873053573 who asked a nearby local what the creatures were called 
453 0.030289126640769135 what are four species that are commonly referred to as kangaroos 
454 0.013332148306149432 are kangaroos and wallabies adept swimmers 
455 0.011903918403013953 are kangaroos farmed to any extent 
456 0.4472135954999579 who also discovered that less than three percent of kangaroos exposed to the virus developed blindness  
457 0.4472135954999579 different species of kangaroos eat what 
458 0.4472135954999579 have kangaroos dazzled by headlights or startled by engine noise been known to leap in front of cars  
45

571 0.8320502943378437 what is one of the challenges of re establishing a population of eurasian otters in the uk 
572 0.4472135954999579 is otter a kind of mammal 
573 0.047565149415449405 how many species and genera does otter have 
574 0.01999600119960014 do otters live in water 
575 0.004672846178479892 where do sea otters live 
576 0.1643989873053573 where does the word  otter  derive from 
577 0.004405243598534552 does otter give birth or lay egg 
578 0.030289126640769135 what drives sea otter almost to extinction 
579 0.016126934718260072 why otters are considered as totem animals 
580 0.1643989873053573 do sea otters live along the pacific coast 
581 0.004830861503963699 are otters totem animals 
582 0.004830861503963699 are otters herbivores 
583 0.1643989873053573 what is the primary item in an otter s diet 
584 0.04163054471218133 what is an otter s den called 
585 0.1643989873053573 why is the giant otter becoming increasingly rare 
586 0.047565149415449405 how do otters ke

...

1689 0.4472135954999579 did he become chief engineer in the department of bridges and highways in 1892 
1690 0.12403473458920847 was becquerel a french physicist 
1691 0.1643989873053573 whom did he share the nobel prize with 
1692 0.1643989873053573 what is the si unit for radioactivity called 
1693 0.08304547985373997 when becquerel discovered radioactivity  whose work did he investigate 
1694 0.007936257995306542 where are there craters named becquerel 
1695 0.1643989873053573 why did bequerel win the nobel prize in physics 
1696 0.12403473458920847 was isaac newton british 
1697 0.12403473458920847 was isaac newton religious 
1698 0.08304547985373997 did isaac newton die in 1898 
1699 0.12403473458920847 where was isaac newton born 
1700 0.02630668208823282 who shares credit with isaac newton for developing calculus 
1701 0.1643989873053573 when did isaac newton discover the generalized binomial theorem 
1702 0.4472135954999579 did the occult influence newton s theory of gravitatio

1797 0.1643989873053573 is it a host city for the formula one world championship 
1798 0.4472135954999579 is the central market located in the proximity of the pertama complex 
1799 0.4472135954999579 is it home to the parliament of malaysia 
1800 0.4472135954999579 when did kuala lumpur become a federal territory of malaysia 
1801 0.1643989873053573 what is the city s population 
1802 0.31622776601683794 what is the official religion in the country 
1803 0.029399051601892736 is there a more industrialized region in malaysia 
1804 0.31622776601683794 after the 2008 elections  what party had the most parliament seats 
1805 0.4472135954999579 what is the name of the event that draws the worlds top riders to malaysia 
1806 0.016126934718260072 wasn t leonardo da vinci valued as an engineer 
1807 0.023802777946288958 wasn t leonardo da vinci born on april 15 
1808 0.04163054471218133 who left verroccio s studio 
1809 0.4472135954999579 when did baldassare castiglione  author of il cortegia

1927 0.12403473458920847 was tesla an american citizen 
1928 0.4472135954999579 was tesla on the cover of time magazine 
1929 0.08304547985373997 when did tesla receive his last patent 
1930 0.08304547985373997 what languages did tesla speak 
1931 0.1643989873053573 did tesla win the nobel prize 
1932 0.4472135954999579 was tesla rich at the time of his death 
1933 0.12403473458920847 what music group was named after tesla 
1934 0.06237828615518053 consequently  what is not considered a  serious painter  by some contemporary artists  who often regard his work as bourgeois and kitsch 
1935 0.1643989873053573 who spent the next 10 years painting for look magazine 
1936 0.1643989873053573 when was his last painting for the post published 
1937 0.08304547985373997 when did rockwell marry his first wife  irene o connor  
1938 0.4472135954999579 did norman rockwell spend the winter months as artist in residence at otis college of art as well as design 
1939 0.4472135954999579 did norman rock

2039 0.04163054471218133 what is northern california s most widely circulated newspaper
2040 0.1643989873053573 what makes san francisco among the top ten north american destinations for conventions and conferences
2041 0.02630668208823282 who manages candlestick point
2042 0.01281945932506583 is crissy field an airfield
2043 0.31622776601683794 does the de young museum house the asian art museum
2044 0.1643989873053573 what prompted the city to upgrade its building codes
2045 0.4472135954999579 like many larger where cities  san francisco is a minority majority city  as non hispanic whites comprise less than half of the population 
2046 0.1643989873053573 who laid out a street plan for the expanded s10 settlement 
2047 0.12403473458920847 when was a 43 acre mission bay campus opened 
2048 0.1643989873053573 when did it sign on the air 
2049 0.4472135954999579 give an example of the largest two year community colleges in the country 
2050 0.4472135954999579 give an example of the ten b

2148 0.029399051601892736 does a violin have four strings 
2149 0.31622776601683794 is the violin supported by the left shoulder 
2150 0.1643989873053573 is the violin shaped like an hourglass 
2151 0.4472135954999579 what were strings first made out of 
2152 0.030289126640769135 what is a violin called informally 
2153 0.030289126640769135 what does vitula mean 
2154 0.030289126640769135 what is a person that makes or repairs violins called 
2155 0.4472135954999579 what does the  voice  of a violin depend upon 
2156 0.1643989873053573 is the violin a percussion instrument 
2157 0.030289126640769135 what happened in 1980s 
2158 0.02630668208823282 who features violinist boyd tinsley 
2159 0.08304547985373997 regarding this topic  what did antonio stradivari do 
2160 0.08304547985373997 when did charles ix order amati to construct 24 violins for him 
2161 0.058722021951470346 is a string first tuned to a standard pitch 
2162 0.4472135954999579 is each node at an integer division of the 